# IMPORTS

## Libraries

In [20]:
import warnings

import numpy  as np
import pandas as pd

import seaborn           as sns
import matplotlib.pyplot as plt

from scipy import stats as ss

from IPython.display      import Image
from IPython.core.display import HTML

from sklearn.preprocessing   import RobustScaler, MinMaxScaler
from sklearn.model_selection import train_test_split


warnings.filterwarnings("ignore")

## Load Dataset

In [21]:
dfRaw = pd.read_feather('00-Data/FeatherData/df04.feather')

In [22]:
dfRaw.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,AgeGroup,Origin
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,True,True,101348.88,True,Adult,Latin
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,False,True,112542.58,False,Adult,Latin
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,True,False,113931.57,True,Adult,Latin
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,False,False,93826.63,False,Adult,Latin
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,True,True,79084.10,False,Adult,Latin


## Helper Functions

#### Jupyter Settings

In [23]:
def jupyter_settings():
    %matplotlib inline
    %pylab inline
    
    plt.style.use('bmh')
    plt.rcParams['figure.figsize'] = [25, 12]
    plt.rcParams['font.size'] = 24
    
    display( HTML('<style>.container { width:100% !important; }</style>'))
    pd.options.display.max_columns = None
    pd.options.display.max_rows = None
    pd.set_option('display.expand_frame_repr', False)
    
    seed = 0
    np.random.seed(seed)
        
    sns.set()

In [24]:
jupyter_settings()  

Populating the interactive namespace from numpy and matplotlib


#### Functions

# DATA PREPARATION

In [25]:
df05 = dfRaw.copy()

## Change Dtypes

In [26]:
df05[df05.select_dtypes(include=['bool']).columns] = df05.select_dtypes(include=['bool']).astype(int)

## Columns Filter 

In [27]:
#Categorical Attributes
toDrop = ['Surname', 'RowNumber', 'CustomerId']
df05 = df05.drop(toDrop, axis=1)

## Transformation

### Encoding

In [28]:
#AgeGroup -> One Hot Encoding
df05 = pd.get_dummies(df05, prefix=['AgeGroup'], columns=['AgeGroup'], drop_first=True)

#Geography -> One Hot Encoding
df05 = pd.get_dummies(df05, prefix=['Geography'], columns=['Geography'], drop_first=True)

# Gender
df05['Gender'] = np.where(df05['Gender'] == 'Female', 1, 0)

# origin
df05['Origin'] = np.where(df05['Origin'] == 'Latin', 1, 0)

## Data Split

In [29]:
X = df05.drop(columns=['Exited'], axis=1)
Y = df05['Exited']

# spliting into test dataset
XTrain, XTest, yTrain, yTest = train_test_split(X, Y, test_size=0.1, random_state=42, stratify=Y)
testDataset = pd.concat([XTest, yTest], axis=1).reset_index()
testDataset.to_feather('00-Data/FeatherData/testDataset.feather')

## ~~Normalization~~

## Rescaling

In [30]:
# Numerical Attributes
numAttributes = XTrain.select_dtypes(include=['int32', 'int64', 'float64'])
numAttributes.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Origin
673,640,1,32,6,118879.35,2,1,1,19131.71,1
7709,598,1,64,1,62979.93,1,1,1,152273.57,1
6100,796,0,30,2,137262.71,2,1,0,62905.29,1
8469,720,0,71,5,183135.39,2,1,1,197688.50,1
8563,578,0,45,1,148600.91,1,1,0,143397.14,1


In [31]:
rs = RobustScaler()
mms = MinMaxScaler()

#Balance >> Presence of well defined outiliers
XTrain['Balance'] = rs.fit_transform(XTrain[['Balance']].values)
testDataset['Balance'] = rs.transform(testDataset[['Balance']].values)

#CreditScore
XTrain['CreditScore'] = mms.fit_transform(XTrain[['CreditScore']].values)
testDataset['CreditScore'] = rs.transform(testDataset[['CreditScore']].values)


#Age
XTrain['Age'] = mms.fit_transform(XTrain[['Age']].values)
testDataset['Age'] = rs.transform(testDataset[['Age']].values)

#Tenure
XTrain['Tenure'] = mms.fit_transform(XTrain[['Tenure']].values)
testDataset['Tenure'] = rs.transform(testDataset[['Tenure']].values)

#NumOfProducts
XTrain['NumOfProducts'] = mms.fit_transform(XTrain[['NumOfProducts']].values)
testDataset['NumOfProducts'] = rs.transform(testDataset[['NumOfProducts']].values)

#EstimatedSalary
XTrain['EstimatedSalary'] = mms.fit_transform(XTrain[['EstimatedSalary']].values)
testDataset['EstimatedSalary'] = rs.transform(testDataset[['EstimatedSalary']].values)

# Convert to .feather

In [32]:
#Train Dataset
trainDataset = pd.concat([XTrain, yTrain], axis=1).reset_index()
trainDataset.to_feather('00-Data/FeatherData/trainDatasetScaling.feather')

#Test Dataset
testDataset.to_feather('00-Data/FeatherData/testDatasetScaling.feather')